In [423]:
import numpy as np
import tensorflow as tf
from keras import backend as kb

def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
    """
    Arguments:
    box_confidence -- tensor of shape (S, S, B, 1)
    boxes -- tensor of shape (S, S, B, 4)
    box_class_probs -- tensor of shape (S, S, B, C)
    threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box

    S - number of grid, B - number of anchor boxes, C - classes

    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes

    Note: "None" is here because you don't know the exact number of selected boxes, as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.    
    """

    box_scores = tf.multiply(box_confidence, box_class_probs)    # confidence broadcasted
    box_max_classes = tf.argmax(box_scores, axis=-1)
    box_max_scores = kb.max(box_scores, axis=-1)
    
    mask = box_max_scores >= threshold
    
    scores = tf.boolean_mask(box_max_scores, mask)
    boxes = tf.boolean_mask(boxes, mask)
    classes = tf.boolean_mask(box_max_classes, mask)
    
    return scores, boxes, classes
    
    
def iou(pivot_box, boxes):
    """
    Arguments:
    pivot_box -- one box, tensor of shape (4) (x1, y1, x2, y2)
    boxes -- comparing boxes, tensor of shape (None, 4) (x1, y1, x2, y2)
    
    Returns:
    ious -- tensor of shape (None, 1), iou values 
    """        
    ix = tf.subtract(tf.minimum(pivot_box[2], boxes[:, 2]), tf.maximum(pivot_box[0], boxes[:, 0]))
    iy = tf.subtract(tf.minimum(pivot_box[3], boxes[:, 3]), tf.maximum(pivot_box[1], boxes[:, 1]))
    
    intersect = tf.multiply(tf.maximum(ix, 0.), tf.maximum(iy, 0.))
    b1 = tf.multiply(tf.subtract(pivot_box[2], pivot_box[0]), tf.subtract(pivot_box[3], pivot_box[1]))
    b2 = tf.multiply(tf.subtract(boxes[:, 2], boxes[:, 0]), tf.subtract(boxes[:, 3], boxes[:, 1]))
    union = tf.subtract(tf.add(b1, b2), intersect)
    
    return tf.divide(intersect, union)
    
    
def non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None,), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. 
    """
    
    """
    tscores = tf.Variable(scores)
    tboxes = tf.Variable(boxes)
    tf.get_default_session().run(tf.variables_initializer([tscores, tboxes]))
    """    
    #nms_list = []
    nms_indices = tf.constant([], dtype=tf.int64)
    z_score = tf.zeros_like(scores)
    z_box = tf.zeros_like(boxes)
    
#    max_boxes_tensor = tf.Variable(max_boxes)
#    tf.get_default_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    def cond(tscores, tboxes, indices):
        return tf.logical_not(tf.reduce_all(tf.equal(tscores, z_score)))
    
    def body(tscores, tboxes, indices):
        # find the box with maximum score
        maxidx = tf.argmax(tscores)
        indices = tf.concat([indices, [maxidx]], axis=-1)
        
        # mark the rests that have less scores than iou_threshold
        iou_mat = iou(tboxes[maxidx], tboxes)
        mask = tf.less_equal(iou_mat, iou_threshold)

        tscores = tf.where(mask, tscores, z_score)
        tboxes = tf.where(mask, tboxes, z_box)
        
        return [tscores, tboxes, indices]
 
    final = tf.while_loop(cond, body, loop_vars=[scores, boxes, nms_indices],\
                          shape_invariants=[scores.get_shape(), boxes.get_shape(), tf.TensorShape([None, ])], \
                          maximum_iterations=max_boxes)
                
    scores = kb.gather(scores, final[2])
    boxes = kb.gather(boxes, final[2])
    classes = kb.gather(classes, final[2])
    
    return scores, boxes, classes

In [424]:

def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. Note also that this
    function will transpose the shapes of scores, boxes, classes. This is made for convenience.
    """
    
    max_boxes_tensor = kb.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    kb.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
    ### START CODE HERE ### (≈ 1 line)
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold)
    ### END CODE HERE ###
    
    print(nms_indices.eval())
    
    # Use K.gather() to select only nms_indices from scores, boxes and classes
    ### START CODE HERE ### (≈ 3 lines)
    scores = kb.gather(scores, nms_indices)
    boxes = kb.gather(boxes, nms_indices)
    classes = kb.gather(classes, nms_indices)
    ### END CODE HERE ###
    
    return scores, boxes, classes

In [425]:
with tf.Session() as sess:
    
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    """
    scores = tf.constant([1, 2, 3, 4, 5, 1], dtype=tf.float32)
    boxes = tf.constant([[0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 2, 1], [4, 4, 5, 6], [4, 4, 5, 5], [6, 6, 7, 10]], dtype=tf.float32)
    classes = tf.constant([1, 1, 1, 1, 1, 2], dtype=tf.float32)
    #sess.run(tf.global_variables_initializer())
    """
    print("iou = " + str(iou(boxes[4], boxes).eval()))
    
    scores, boxes, classes = non_max_suppression(scores, boxes, classes, iou_threshold=0.4, max_boxes=10)

    """
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    """
    s, b, c = sess.run ([scores, boxes, classes])
    print("scores = " + str(s))
    print("boxes = " + str(b))
    print("classes = " + str(c))
    
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

iou = [-0. -0. -0.  0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  0. -0. -0. -0.  0.
 -0. -0. -0. -0.  0. -0. -0. -0. -0.  0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
scores = [ 8.42653275  8.42653275  8.42653275  8.42653275  8.42653275  8.42653275
  8.42653275  8.42653275  8.42653275  8.42653275]
boxes = [[ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]
 [ 5.73276615 -0.54868162 -9.347929    0.58930719]]
classes = [ 8.42653275  8.42653275  8.42653275  8.42653275  8.42653275  8.42653275
  8.42653275  8.42653275  8.42653275  8.

In [158]:
S = 19
with tf.Session() as test_a:
    box_confidence = tf.random_normal([S, S, 5, 1], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([S, S, 5, 4], mean=1, stddev=4, seed = 1)
    box_class_probs = tf.random_normal([S, S, 5, 80], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

   # scores, boxes, classes = non_max_suppression(scores, boxes, classes, 10)

    print("iou = " + str(iou(box1, box2).eval()))

scores[2] = 10.7506
boxes[2] = [ 8.42653275  3.27136683 -0.53134358 -4.94137335]
classes[2] = 7
scores.shape = (?,)
boxes.shape = (?, 4)
classes.shape = (?,)


NameError: name 'box1' is not defined

In [164]:
def dist(a, b):
    return tf.add(tf.multiply(a[0], b[0]), tf.multiply(a[1], b[1]))

with tf.Session() as test_a:
    a = np.array([[1, 1], [2, 2], [3, 1]])
    aa = np.array([1, 4])
    b = tf.zeros_like(a)
    c = tf.where(a > 1)
    print(tf.gather_nd(a, c).eval())
    print(c.eval())
    

[2 2 3]
[[1 0]
 [1 1]
 [2 0]]


In [324]:
with tf.Session() as test_a:
    nms_list = []
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)

    z_score = tf.zeros_like(scores)
    z_box = tf.zeros_like(boxes)
    
    def cond(tscores, tboxes):
        return scores == z_score
 
    def body(tscores, tboxes):
        return [z_score, z_box]

    f = tf.while_loop(cond, body, loop_vars=[scores, boxes], maximum_iterations=10)
    print(test_a.run(f))

[array([-2.24527287,  6.93839502,  1.26131749, -8.7708168 ,  1.39699364,
        3.36489725,  3.37129188, -7.49171829, -1.89158893,  0.7749185 ,
        3.57417917, -0.05729628,  8.42653275,  3.27136683, -0.53134358,
       -4.94137335,  6.04708433,  0.8987757 , -0.05851877,  7.13125515,
       -5.97190857, -0.75157177, -1.26403999,  2.28267717,  5.53132391,
       -8.11302853,  2.93124819, -4.25083494,  2.4274013 , -5.92113352,
        0.83932906,  4.59864759, -4.52235651,  6.92584944,  0.01802081,
       -1.93058872,  0.21641421,  1.28683209,  3.53192353, -5.28476286,
        6.31752491, -3.69346809,  1.12617838,  2.9082098 ,  2.74776793,
       -0.26723564, -0.80300128, -6.22426271,  1.4995985 , -2.08261681,
       -1.98496628, -0.12781298, -6.83526182, -0.35044277], dtype=float32), array([[ -2.24527287,   6.93839502,   1.26131749,  -8.7708168 ],
       [  1.39699364,   3.36489725,   3.37129188,  -7.49171829],
       [ -1.89158893,   0.7749185 ,   3.57417917,  -0.05729628],
       [

In [401]:
with tf.Session() as test_a:
    nms_list = []
    i = tf.constant(0)
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)

    print(scores.eval())
    
    nms_indices = tf.constant([], dtype=tf.int64)
    z_score = tf.zeros_like(scores)
    z_box = tf.zeros_like(boxes)
    
#    max_boxes_tensor = tf.Variable(max_boxes)
#    tf.get_default_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    def cond(tscores, tboxes, indices):
        return tf.reduce_all(tf.not_equal(tscores, z_score))
    
    def body(tscores, tboxes, indices):
        # find the box with maximum score
        maxidx = tf.argmax(tscores)
        indices = tf.concat([indices, [maxidx]], axis=-1)
        
        # mark the rests that have less scores than iou_threshold
        iou_mat = iou(tboxes[maxidx], tboxes)
        mask = tf.less_equal(iou_mat, 0.5)

        tscores = tf.where(mask, tscores, z_score)
        tboxes = tf.where(mask, tboxes, z_box)
        
        return [tscores, tboxes, indices]
 
    final = tf.while_loop(cond, body, loop_vars=[scores, boxes, nms_indices],\
                          shape_invariants=[scores.get_shape(), boxes.get_shape(), tf.TensorShape([None, ])], \
                          maximum_iterations=5)
    
    test_a.run(final)
    print(final[0].eval())
    print(final[1].eval())
    print(final[2].eval())
    

[-2.24527287  6.93839502  1.26131749 -8.7708168   1.39699364  3.36489725
  3.37129188 -7.49171829 -1.89158893  0.7749185   3.57417917 -0.05729628
  8.42653275  3.27136683 -0.53134358 -4.94137335  6.04708433  0.8987757
 -0.05851877  7.13125515 -5.97190857 -0.75157177 -1.26403999  2.28267717
  5.53132391 -8.11302853  2.93124819 -4.25083494  2.4274013  -5.92113352
  0.83932906  4.59864759 -4.52235651  6.92584944  0.01802081 -1.93058872
  0.21641421  1.28683209  3.53192353 -5.28476286  6.31752491 -3.69346809
  1.12617838  2.9082098   2.74776793 -0.26723564 -0.80300128 -6.22426271
  1.4995985  -2.08261681 -1.98496628 -0.12781298 -6.83526182 -0.35044277]
[  5.89251471e+00  -2.66019273e+00   4.21800900e+00  -5.08673906e-01
   4.36911583e+00   9.97112930e-01   8.80744553e+00   1.00348520e+00
   3.88418341e+00  -5.21010399e+00   1.38635576e-01   1.80521035e+00
  -4.00314140e+00   2.50099659e+00   1.25946712e+00   1.76283860e+00
  -2.98894405e-01  -3.72097683e+00  -1.84456849e+00   5.45544767e+0

In [318]:
with tf.Session() as sess:
    nms_list = []
    #scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    scores = tf.constant(1)
    boxes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    p = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    
    z_score = tf.zeros_like(scores)
    z_box = tf.zeros_like(boxes)
    
    def cond(tscores, tboxes, p):
        return tf.not_equal(tscores, z_score)
        
    def body(tscores, tboxes, p):
        return [z_score, z_box, p]

    f = tf.while_loop(cond, body, loop_vars=[scores, boxes, p], maximum_iterations=10)
    print(sess.run(f))

[0, array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.], dtype=float32), array([[ -2.24527287,   6.93839502,   1.26131749,  -8.7708168 ],
       [  1.39699364,   3.36489725,   3.37129188,  -7.49171829],
       [ -1.89158893,   0.7749185 ,   3.57417917,  -0.05729628],
       [  8.42653275,   3.27136683,  -0.53134358,  -4.94137335],
       [  6.04708433,   0.8987757 ,  -0.05851877,   7.13125515],
       [ -5.97190857,  -0.75157177,  -1.26403999,   2.28267717],
       [  5.53132391,  -8.11302853,   2.93124819,  -4.25083494],
       [  2.4274013 ,  -5.92113352,   0.83932906,   4.59864759],
       [ -4.52235651,   6.92584944,   0.01802081,  -1.93058872],
       [  0.21641421,   1.28683209,   3.53192353,  -5.28476286],
       [  6.31752491,  -

In [341]:
with tf.Session() as sess:
    scores = tf.constant(0)
    boxes = tf.constant([])
    def c(i, m): 
        return i < 10
    
    def b(i, m):         
        return [i+1, tf.concat([m, m], axis=-1)]
    
    a, b = tf.while_loop(c, b, loop_vars=[scores, boxes], shape_invariants=[scores.get_shape(), tf.TensorShape([None,])])
    
    print(sess.run([a, b]))

[10, array([], dtype=float32)]


In [410]:
with tf.Session() as sess:
    a = tf.constant([[1, 2, 4]])
    b = tf.constant([[1, 2, 3]])
    print(tf.reduce_all(kb.not_equal(a, b)).eval())
   

False
